# CHO Cell Surface APOL1 IF

Apolipoprotein L-1 (APOL1) is a primate-specific innate immunity gene that provides protection against several protozoan parasites due to its channel forming properties. Recently evolved APOL1 variants, which provide protection against human-infective trypanosomes, termed G1 and G2, but not wild-type G0, increase kidney disease risk when present in two copies. In mammalian cells, high expression of G1/G2 leads to swelling and eventual lysis. However, the mechanism of cell death remains elusive, with multiple disparate pathways reported. The following imaging data collected by Joe Giovinazzo (Hunter College) records transfected Cho cells with G0, G1, and G2 coding variants that have been stained for Nuclei, APOl1, and Calnexin (cell death) over 120 minutes with 30 minute timepoints.

The goal of this analysis is to determine the count and intensity of APOL1 expressing cells in each condition at each timepoint. Each channel is therefore segmented and the APOL1 channel signal is summed for each cell individually.

## Imports

In [ ]:
%cd "CHO-Cell-Surface-APOL1-IF"

## Load Data

In [4]:
%cd "C:\Users\paz2009\Documents\Experiments\011 APOL1 (Joe) (Hunter)\CHO RUSH Image Analysis 02-20-19\CHO_RUSH_3"

# Read in data from .tif files

# Import and concatenate images in 3D numpy matrices
data = skimage.io.ImageCollection('*m[0-9][0-9].tif', load_func=imread_rgb)
nuc_data = skimage.io.ImageCollection('*405_ORG.tif', load_func=imread_rgb)
canx_data = skimage.io.ImageCollection('*488_ORG.tif', load_func=imread_rgb)
apol1_data = skimage.io.ImageCollection('*647_ORG.tif', load_func=imread_rgb)
# Images are 20x 512x512 tiles which can be concatenated for convenience
data = skimage.io.concatenate_images(data)
nuc = skimage.io.concatenate_images(nuc_data)
canx = skimage.io.concatenate_images(canx_data)
apol1 = skimage.io.concatenate_images(apol1_data)

filenames = glob.glob('*m[0-9][0-9].tif')
filenames.sort(key=natural_keys) # Note the natural sorting definition so that 
                                 # filenames are in line with ImageCollection Images

C:\Users\paz2009\Documents\Experiments\011 APOL1 (Joe) (Hunter)\CHO RUSH Image Analysis 02-20-19\CHO_RUSH_3


## Define Experimental Factors

In [3]:
# Define conditions and timepoints

cond_types = ['untransfected', 'G0', 'G1', 'G2']
time_types = ['untransfected', 'no', '30min', '60min', '90min', '120min']
rep_types = ['1', '2', '3', '4', '5']
steps = 1

In [5]:
# Creating factors for later analysis
cond = factor_maker(cond_types, filenames, steps)
timepoint = factor_maker(time_types, filenames, steps)
rep = [re.search(r'[0-9]_well', file).group(0)[0] for file in filenames]

## Image Analysis

In [6]:
%cd "C:\Users\paz2009\Documents\Experiments\011 APOL1 (Joe) (Hunter)\Image Analysis 2-20-19\joe_output\Nuclei"

# Define nuclei through local thresholding. Filter out erroneously identified 
# nuclei via a size filter. Dilate nuclei to approximate cell shape through 
# morphological dilation. Watershed and clear borders.

nuc_org = []
nuc_2 = []
w_lines = []

for n, image in enumerate(nuc):
    thresh = threshold_local(image, 599)
    # A local thresholding method is utilized to capture nuclei despite variability of DAPI stain
    binary = image > thresh
    
    binary_label, n_nuc = mh.label(binary)
    nuc_sizes = mh.labeled.labeled_size(binary_label)
    not_nuc = np.where(nuc_sizes < 80) # An initial size filter to filter noise
    binary = mh.labeled.remove_regions(binary_label, not_nuc)
    
    nuc_org.append(clear_border(binary))
    
    binary_dilation = skimage.morphology.dilation(binary, selem) # Binary dilation to mimic cell shape
    cleared = clear_border(binary_dilation)
    
    distance = sp.ndimage.distance_transform_edt(cleared)
    distance = mh.stretch(distance)
    surface = distance.max() - distance

    maxima = mh.regmax(distance, Bc)

    spots, n_spots = mh.label(maxima, Bc=Bc)
    surface = (distance.max() - distance)

    res = mh.cwatershed(surface, spots)
    res *= cleared
    
    ws_nuc_sizes = mh.labeled.labeled_size(res)
    ws_small_nuc = np.where(ws_nuc_sizes < 110) 
    ws_large_nuc = np.where(ws_nuc_sizes > 700)
    # Secondary size filters to filter improperly watershed nuclei
    ws_nuc = mh.labeled.remove_regions(res, ws_small_nuc)
    ws_nuc2 = mh.labeled.remove_regions(ws_nuc, ws_large_nuc)

    re_ws_nuc2, n_left = mh.labeled.relabel(ws_nuc2)
    
    nuc_2.append(re_ws_nuc2)

    # Exporting images to verify method
    fig, ax = plt.subplots(2, 1, figsize=(10, 20))

    ax[0].imshow(image, cmap='gray')
    ax[0].set_title('Image', fontsize = 28)
    ax[0].axis('off')
    
    ax[1].imshow(ws_nuc2, cmap=rmap)
    ax[1].set_title('Watershed', fontsize = 28)
    ax[1].axis('off')

    plt.savefig("nuclei %s %s %s" % (cond[n], timepoint[n], n), dpi=75)
    plt.close()
    
nuc_2 = skimage.io.concatenate_images(nuc_2)

[WinError 3] The system cannot find the path specified: 'C:\\Users\\paz2009\\Documents\\Experiments\\011 APOL1 (Joe) (Hunter)\\Image Analysis 2-20-19\\joe_output\\Nuclei'
C:\Users\paz2009\Documents\Experiments\011 APOL1 (Joe) (Hunter)\CHO RUSH Image Analysis 02-20-19\CHO_RUSH_3


2022it [11:47,  2.80it/s]


In [2]:
%cd "C:\Users\paz2009\Documents\Experiments\011 APOL1 (Joe) (Hunter)\Image Analysis 2-20-19\joe_output\APOL1"

# Reduction of APOL1 channel noise by subtracting mean channel
# intensity from the channel.

apol1_2 = []

for n, image in enumerate(apol1):
    denoise = image - np.mean(image)
    apol1_min = denoise.clip(0)
    
    apol1_2.append(apol1_min)

    # Exporting images to verify method
    fig, ax = plt.subplots(2, 1, figsize=(20, 30))

    ax[0].imshow(image, cmap='gray')
    ax[0].set_title('Image', fontsize = 28)
    ax[0].axis('off')
    
    ax[1].imshow(apol1_min, cmap='gray')
    ax[1].set_title('Grayscale Threshold', fontsize = 28)
    ax[1].axis('off')
    
    plt.savefig("APOL1 %s %s %s" % (cond[n], timepoint[n], n), dpi=100)
    plt.close()

apol1_2 = skimage.io.concatenate_images(apol1_2)

[WinError 3] The system cannot find the path specified: 'C:\\Users\\paz2009\\Documents\\Experiments\\011 APOL1 (Joe) (Hunter)\\Image Analysis 2-20-19\\joe_output\\APOL1'
C:\Users\paz2009\CV\notebooks


NameError: name 'apol1' is not defined

In [8]:
%cd "C:\Users\paz2009\Documents\Experiments\011 APOL1 (Joe) (Hunter)\Image Analysis 2-20-19\joe_output\Calnexin"

# Threshold the calnexin signal

canx_2 = []

for n, image in enumerate(canx):
    
    denoise = skimage.filters.rank.median(image, disk(6))
    thresh = threshold_otsu(denoise, nbins=256)
    image_sub = denoise - np.median(denoise)
    binary = image_sub > thresh
    
    binary_label, n_spots = mh.label(binary)
    
    spot_sizes = mh.labeled.labeled_size(binary_label)
    small_spots = np.where(spot_sizes < 10)
    filt_spots = mh.labeled.remove_regions(binary_label, small_spots)

    spots_label, n_left = mh.labeled.relabel(filt_spots)
    
    can_bool = spots_label > 0
    
    canx_2.append(can_bool)
    
    # Exporting images to verify method
    fig, ax = plt.subplots(2, 1, figsize=(15, 25))

    ax[0].imshow(image, cmap='gray')
    ax[0].set_title('Image', fontsize = 28)
    ax[0].axis('off')

    ax[1].imshow(spots_label, cmap=rmap)
    ax[1].set_title('Otsu Threshold', fontsize = 28)
    ax[1].axis('off')

    plt.savefig("Calnexin %s %s %s" % (cond[n], timepoint[n], n), dpi=100)
    plt.close()

canx_2 = skimage.io.concatenate_images(canx_2)

[WinError 3] The system cannot find the path specified: 'C:\\Users\\paz2009\\Documents\\Experiments\\011 APOL1 (Joe) (Hunter)\\Image Analysis 2-20-19\\joe_output\\Calnexin'
C:\Users\paz2009\Documents\Experiments\011 APOL1 (Joe) (Hunter)\CHO RUSH Image Analysis 02-20-19\CHO_RUSH_3


C:\Users\paz2009\Anaconda3\envs\CV\lib\site-packages\skimage\filters\rank\generic.py:102: UserWarning: Bitdepth of 15 may result in bad rank filter performance due to large number of bins.
  "performance due to large number of bins." % bitdepth)
C:\Users\paz2009\Anaconda3\envs\CV\lib\site-packages\skimage\filters\rank\generic.py:102: UserWarning: Bitdepth of 15 may result in bad rank filter performance due to large number of bins.
  "performance due to large number of bins." % bitdepth)
C:\Users\paz2009\Anaconda3\envs\CV\lib\site-packages\skimage\filters\rank\generic.py:102: UserWarning: Bitdepth of 12 may result in bad rank filter performance due to large number of bins.
  "performance due to large number of bins." % bitdepth)
C:\Users\paz2009\Anaconda3\envs\CV\lib\site-packages\skimage\filters\rank\generic.py:102: UserWarning: Bitdepth of 13 may result in bad rank filter performance due to large number of bins.
  "performance due to large number of bins." % bitdepth)
C:\Users\paz2009

C:\Users\paz2009\Anaconda3\envs\CV\lib\site-packages\skimage\filters\rank\generic.py:102: UserWarning: Bitdepth of 13 may result in bad rank filter performance due to large number of bins.
  "performance due to large number of bins." % bitdepth)
C:\Users\paz2009\Anaconda3\envs\CV\lib\site-packages\skimage\filters\rank\generic.py:102: UserWarning: Bitdepth of 13 may result in bad rank filter performance due to large number of bins.
  "performance due to large number of bins." % bitdepth)
C:\Users\paz2009\Anaconda3\envs\CV\lib\site-packages\skimage\filters\rank\generic.py:102: UserWarning: Bitdepth of 13 may result in bad rank filter performance due to large number of bins.
  "performance due to large number of bins." % bitdepth)
C:\Users\paz2009\Anaconda3\envs\CV\lib\site-packages\skimage\filters\rank\generic.py:102: UserWarning: Bitdepth of 13 may result in bad rank filter performance due to large number of bins.
  "performance due to large number of bins." % bitdepth)
C:\Users\paz2009

C:\Users\paz2009\Anaconda3\envs\CV\lib\site-packages\skimage\filters\rank\generic.py:102: UserWarning: Bitdepth of 15 may result in bad rank filter performance due to large number of bins.
  "performance due to large number of bins." % bitdepth)
C:\Users\paz2009\Anaconda3\envs\CV\lib\site-packages\skimage\filters\rank\generic.py:102: UserWarning: Bitdepth of 15 may result in bad rank filter performance due to large number of bins.
  "performance due to large number of bins." % bitdepth)
C:\Users\paz2009\Anaconda3\envs\CV\lib\site-packages\skimage\filters\rank\generic.py:102: UserWarning: Bitdepth of 14 may result in bad rank filter performance due to large number of bins.
  "performance due to large number of bins." % bitdepth)
C:\Users\paz2009\Anaconda3\envs\CV\lib\site-packages\skimage\filters\rank\generic.py:102: UserWarning: Bitdepth of 14 may result in bad rank filter performance due to large number of bins.
  "performance due to large number of bins." % bitdepth)
C:\Users\paz2009

C:\Users\paz2009\Anaconda3\envs\CV\lib\site-packages\skimage\filters\rank\generic.py:102: UserWarning: Bitdepth of 13 may result in bad rank filter performance due to large number of bins.
  "performance due to large number of bins." % bitdepth)
C:\Users\paz2009\Anaconda3\envs\CV\lib\site-packages\skimage\filters\rank\generic.py:102: UserWarning: Bitdepth of 14 may result in bad rank filter performance due to large number of bins.
  "performance due to large number of bins." % bitdepth)
C:\Users\paz2009\Anaconda3\envs\CV\lib\site-packages\skimage\filters\rank\generic.py:102: UserWarning: Bitdepth of 15 may result in bad rank filter performance due to large number of bins.
  "performance due to large number of bins." % bitdepth)
C:\Users\paz2009\Anaconda3\envs\CV\lib\site-packages\skimage\filters\rank\generic.py:102: UserWarning: Bitdepth of 14 may result in bad rank filter performance due to large number of bins.
  "performance due to large number of bins." % bitdepth)
C:\Users\paz2009

...

C:\Users\paz2009\Anaconda3\envs\CV\lib\site-packages\skimage\filters\rank\generic.py:102: UserWarning: Bitdepth of 14 may result in bad rank filter performance due to large number of bins.
  "performance due to large number of bins." % bitdepth)
C:\Users\paz2009\Anaconda3\envs\CV\lib\site-packages\skimage\filters\rank\generic.py:102: UserWarning: Bitdepth of 13 may result in bad rank filter performance due to large number of bins.
  "performance due to large number of bins." % bitdepth)
C:\Users\paz2009\Anaconda3\envs\CV\lib\site-packages\skimage\filters\rank\generic.py:102: UserWarning: Bitdepth of 11 may result in bad rank filter performance due to large number of bins.
  "performance due to large number of bins." % bitdepth)
C:\Users\paz2009\Anaconda3\envs\CV\lib\site-packages\skimage\filters\rank\generic.py:102: UserWarning: Bitdepth of 12 may result in bad rank filter performance due to large number of bins.
  "performance due to large number of bins." % bitdepth)
C:\Users\paz2009

C:\Users\paz2009\Anaconda3\envs\CV\lib\site-packages\skimage\filters\rank\generic.py:102: UserWarning: Bitdepth of 14 may result in bad rank filter performance due to large number of bins.
  "performance due to large number of bins." % bitdepth)
C:\Users\paz2009\Anaconda3\envs\CV\lib\site-packages\skimage\filters\rank\generic.py:102: UserWarning: Bitdepth of 14 may result in bad rank filter performance due to large number of bins.
  "performance due to large number of bins." % bitdepth)
C:\Users\paz2009\Anaconda3\envs\CV\lib\site-packages\skimage\filters\rank\generic.py:102: UserWarning: Bitdepth of 13 may result in bad rank filter performance due to large number of bins.
  "performance due to large number of bins." % bitdepth)
C:\Users\paz2009\Anaconda3\envs\CV\lib\site-packages\skimage\filters\rank\generic.py:102: UserWarning: Bitdepth of 13 may result in bad rank filter performance due to large number of bins.
  "performance due to large number of bins." % bitdepth)
C:\Users\paz2009

C:\Users\paz2009\Anaconda3\envs\CV\lib\site-packages\skimage\filters\rank\generic.py:102: UserWarning: Bitdepth of 14 may result in bad rank filter performance due to large number of bins.
  "performance due to large number of bins." % bitdepth)
C:\Users\paz2009\Anaconda3\envs\CV\lib\site-packages\skimage\filters\rank\generic.py:102: UserWarning: Bitdepth of 12 may result in bad rank filter performance due to large number of bins.
  "performance due to large number of bins." % bitdepth)
C:\Users\paz2009\Anaconda3\envs\CV\lib\site-packages\skimage\filters\rank\generic.py:102: UserWarning: Bitdepth of 12 may result in bad rank filter performance due to large number of bins.
  "performance due to large number of bins." % bitdepth)
C:\Users\paz2009\Anaconda3\envs\CV\lib\site-packages\skimage\filters\rank\generic.py:102: UserWarning: Bitdepth of 13 may result in bad rank filter performance due to large number of bins.
  "performance due to large number of bins." % bitdepth)
C:\Users\paz2009

C:\Users\paz2009\Anaconda3\envs\CV\lib\site-packages\skimage\filters\rank\generic.py:102: UserWarning: Bitdepth of 15 may result in bad rank filter performance due to large number of bins.
  "performance due to large number of bins." % bitdepth)
C:\Users\paz2009\Anaconda3\envs\CV\lib\site-packages\skimage\filters\rank\generic.py:102: UserWarning: Bitdepth of 14 may result in bad rank filter performance due to large number of bins.
  "performance due to large number of bins." % bitdepth)
C:\Users\paz2009\Anaconda3\envs\CV\lib\site-packages\skimage\filters\rank\generic.py:102: UserWarning: Bitdepth of 13 may result in bad rank filter performance due to large number of bins.
  "performance due to large number of bins." % bitdepth)
C:\Users\paz2009\Anaconda3\envs\CV\lib\site-packages\skimage\filters\rank\generic.py:102: UserWarning: Bitdepth of 12 may result in bad rank filter performance due to large number of bins.
  "performance due to large number of bins." % bitdepth)
C:\Users\paz2009

C:\Users\paz2009\Anaconda3\envs\CV\lib\site-packages\skimage\filters\rank\generic.py:102: UserWarning: Bitdepth of 15 may result in bad rank filter performance due to large number of bins.
  "performance due to large number of bins." % bitdepth)
C:\Users\paz2009\Anaconda3\envs\CV\lib\site-packages\skimage\filters\rank\generic.py:102: UserWarning: Bitdepth of 13 may result in bad rank filter performance due to large number of bins.
  "performance due to large number of bins." % bitdepth)
C:\Users\paz2009\Anaconda3\envs\CV\lib\site-packages\skimage\filters\rank\generic.py:102: UserWarning: Bitdepth of 12 may result in bad rank filter performance due to large number of bins.
  "performance due to large number of bins." % bitdepth)
C:\Users\paz2009\Anaconda3\envs\CV\lib\site-packages\skimage\filters\rank\generic.py:102: UserWarning: Bitdepth of 14 may result in bad rank filter performance due to large number of bins.
  "performance due to large number of bins." % bitdepth)
C:\Users\paz2009

In [9]:
# Filter cells with colocalized calnexin signal

nuc_3 = []

for i in range(0, len(nuc_2)):
    nuc_2_filt = np.unique(nuc_2[i][canx_2[i]])
    for j in range(0, len(nuc_2_filt)):
        nuc_2[i][nuc_2[i] == nuc_2_filt[j]] = 0
    nuc_2[i], n_left = mh.labeled.relabel(nuc_2[i])
    nuc_3.append(nuc_2[i])

nuc_3 = skimage.io.concatenate_images(nuc_3)

In [10]:
# Sum APOL1 intensity per cell for all cells

apol1_percell = []
apol1_perimage = []
tile = []

for i in range(0, len(nuc_3)):
    for j in range(1, np.max(nuc_3[i])):
        m = nuc_3[i] == j
        cell_int = np.sum(apol1_2[i][m])
        apol1_percell.append(cell_int)
    cond_cell = pd.Series([cond[i]] * np.max(nuc_3[i]))
    time_cell = pd.Series([timepoint[i]] * np.max(nuc_3[i]))
    rep_cell = pd.Series([rep[i]] * np.max(nuc_3[i]))
    apol1_cell = pd.Series(apol1_percell[:])
    total = pd.concat([cond_cell, time_cell, rep_cell, apol1_cell], axis=1)
    apol1_perimage.append(total)
    apol1_percell.clear()

In [11]:
# Create dataframe of result

result = pd.concat(apol1_perimage, axis=0)
result.columns = ['Condition', 'Timepoint', 'Replicate', 'Intensity']
result['Intensity'] = result['Intensity']/(max(result['Intensity']))

result.to_excel("results.xlsx")